# Visualize pymoo result
Load the pymoo result object saved with pickle and visualize it.

ONOSControllerPlacement needs to be definded to load the result object.
So, execute the following cell first.

In [1]:
#!/usr/bin/env python
import numpy as np
import networkx as nx
import math
import pickle
from pymoo.core.problem import ElementwiseProblem

class ONOSControllerPlacement(ElementwiseProblem):
    def __init__(self, num_nodes, distance_matrix, shortest_paths, graph, **kwargs):
        super().__init__(n_var=2*num_nodes, 
                         n_obj=4, 
                         n_constr=2, 
                         xl=0, xu=1, 
                         **kwargs)
        self.num_nodes = num_nodes
        self.distance_matrix = distance_matrix
        self.shortest_paths = shortest_paths
        self.graph = graph
    
    def _evaluate(self, x, out, *args, **kwargs):
        controller_nodes = x[:self.num_nodes]   # first half is controller placement
        atomix_nodes = x[self.num_nodes:]       # second half is atomix placement


        num_controller = np.sum(controller_nodes)
        num_atomix = np.sum(atomix_nodes)

        # Obj1: Minimize number of contrtoller
        f1 = num_controller

        # Obj2: Minimize number of atomix
        f2 = num_atomix

        # Obj3: Minimize average FSP
        f3 = calculate_FST(self.num_nodes, 
                           controller_nodes, 
                           atomix_nodes, 
                           self.distance_matrix, 
                           self.shortest_paths)
        
        f4 = calculate_BC(self.num_nodes, 
                           controller_nodes, 
                           atomix_nodes, 
                           self.distance_matrix, 
                        #    self.shortest_paths,
                           self.graph)

        # Constr1: The number of controller is equal to or greater than 2
        g1 = 2 - num_controller

        # Constr2: The number of atomix is equal to or greater than 3
        g2 = 3 - num_atomix
        
        # Add the centrality metrix into optimazing objectives:
        # 1. Nearest controller for each switch
        # 2. The number of controlled switches for each controller should be <= limit_num_switches_controlled (limit_num_switches_controlled=int(math.ceil(num_nodes/num_controller)))
        # 3. return value should be the variance for all controller's betweenness centrality
        out["F"] = [f1, f2, f3, f4]
        out["G"] = [g1, g2]


def calculate_FST(num_nodes, controller_nodes, atomix_nodes, distance_matrix, shortest_paths):
    num_controller = np.sum(controller_nodes)
    num_atomix = np.sum(atomix_nodes)
    controller_list = np.nonzero(controller_nodes)[0].tolist()
    atomix_list = np.nonzero(atomix_nodes)[0].tolist()

    if(num_controller == 0 or num_atomix ==0):
        return math.inf

    # find the nearest controller for each switch
    controller_of = []
    for s in range(num_nodes):
        delay = math.inf
        nearest_controller = None
        for c in controller_list:
            if distance_matrix[s][c] < delay:
                delay = distance_matrix[s][c]
                nearest_controller = c
        controller_of.append(nearest_controller)    

    # calculate average delay to atomix nodes from each controller
    average_atomix_delay_from = {}
    for c in controller_list:
        delay = []
        for a in atomix_list:
            delay.append(distance_matrix[c][a])
        average_atomix_delay_from[c] = np.mean(delay)

    # find the nearest atomix for each atomix and calculate average delay
    atomix_atomix_delays = []
    for a1 in atomix_list:
        delay = math.inf
        for a2 in atomix_list:
            if(a1 == a2):
                continue
            if distance_matrix[a1][a2] < delay:
                delay = distance_matrix[a1][a2]
        atomix_atomix_delays.append(delay)
    average_atomix_atomix_delay = np.mean(atomix_atomix_delays)
    FTSs = []
    for source in range(num_nodes):
        for distination in range(num_nodes):
            if(source == distination):
                continue
            delay = 0
            is_controlled_by_single_controller = True
            counted_controllers = []
            for s in shortest_paths[source][distination]:
                # switch-controller delay
                delay += distance_matrix[s][controller_of[s]] * 4

                # controller-atomix delay
                if(s == source):
                    delay += average_atomix_delay_from[controller_of[s]] * 2
                elif(s != distination):
                    if(controller_of[s] != controller_of[source]):
                        is_controlled_by_single_controller = False
                        if(not controller_of[s] in counted_controllers):
                            counted_controllers.append(controller_of[s])
                            delay += average_atomix_delay_from[controller_of[s]]
                else:
                    if(controller_of[s] == controller_of[source]):
                        if(not is_controlled_by_single_controller):
                            delay += average_atomix_delay_from[controller_of[s]]
                    else:
                        delay += average_atomix_delay_from[controller_of[s]] * 2
            
            # atomix-atomix delay
            delay +=  average_atomix_atomix_delay * 2
            FTSs.append(delay)

    return np.mean(FTSs)



def calculate_BC(num_nodes, controller_nodes, atomix_nodes, distance_matrix, graph):
    G = nx.Graph()
    for node1 in range(len(graph)):
        G.add_node(str(node1))
        for node2, delay in graph[node1].items():
            G.add_edge(str(node1), str(node2), weight=delay)
    
    # The list of betweenness centrality for all switches
    nodes_bc=nx.current_flow_betweenness_centrality(G, normalized=True, weight=None, dtype='float', solver='full')
    num_controller = np.sum(controller_nodes)
    num_atomix = np.sum(atomix_nodes)
    controller_list = np.nonzero(controller_nodes)[0].tolist()

    if(num_controller == 0 or num_atomix ==0):
        return math.inf

    # find the nearest controller for each switch
    controller_of = []
    limit_num_switches_controlled=int(math.ceil(num_nodes/num_controller)) # balance the number of switches controllers can control 
    switches_bc_of_controller_ = dict.fromkeys((range(num_nodes)),0) # list of sum of betweenness centrality of switches for each controller
    for s in range(num_nodes):
        delay = math.inf
        nearest_controller = None
        controlled_switches=[]
        for c in controller_list:
            # Conditions: nearest controller (with the lowest delay) && the number of switches for each controller < limit_num_switches_controlled
            if distance_matrix[s][c] < delay and controller_of.count(c) < limit_num_switches_controlled:
                delay = distance_matrix[s][c]
                nearest_controller = c
                controlled_switches.append(s)
        switches_bc_of_controller_[nearest_controller] += nodes_bc[str(s)]
        controller_of.append(nearest_controller)
    
    # Simplify switches_bc_of_controller_ (only need value for calculating variance)
    bc_array = []
    for i in switches_bc_of_controller_.values():
        bc_array.append(i)

    # return variance value can show the degree of balance within all controllers
    return np.var(bc_array)

### Load ctaea results for all topos

In [2]:
with open('res_bc_Cogent_ctaea.pkl','rb') as f_Cogent:
    res_10_Cogent = pickle.load(f_Cogent)
with open('res_bc_UsCarrier_ctaea.pkl','rb') as f_UsCarrier:
    res_10_UsCarrier = pickle.load(f_UsCarrier)
with open('res_bc_HiberniaGlobal_ctaea.pkl','rb') as f_HiberniaGlobal:
    res_10_HiberniaGlobal = pickle.load(f_HiberniaGlobal)
with open('res_bc_Colt_ctaea.pkl','rb') as f_Colt:
    res_10_Colt = pickle.load(f_Colt)
with open('res_bc_Funet_ctaea.pkl','rb') as f_Funet:
    res_10_Funet = pickle.load(f_Funet)
with open('res_bc_Abvt_ctaea.pkl','rb') as f_Abvt:
    res_10_Abvt = pickle.load(f_Abvt)
with open('res_bc_Intellifiber_ctaea.pkl','rb') as f_Intellifiber:
    res_10_Intellifiber = pickle.load(f_Intellifiber)
with open('res_bc_TataNld_ctaea.pkl','rb') as f_TataNld:
    res_10_TataNld = pickle.load(f_TataNld)
with open('res_bc_Internode_ctaea.pkl','rb') as f_Internode:
    res_10_Internode = pickle.load(f_Internode)
with open('res_bc_Missouri_ctaea.pkl','rb') as f_Missouri:
    res_10_Missouri = pickle.load(f_Missouri)
with open('res_bc_Ion_ctaea.pkl','rb') as f_Ion:
    res_10_Ion = pickle.load(f_Ion)
with open('res_bc_Palmetto_ctaea.pkl','rb') as f_Palmetto:
    res_10_Palmetto = pickle.load(f_Palmetto)

## Hypervolume
1. Store values

In [ ]:
# ctaea
F10_Cogent=res_10_Cogent.F
F10_UsCarrier=res_10_UsCarrier.F
F10_HiberniaGlobal=res_10_HiberniaGlobal.F
F10_Colt=res_10_Colt.F
F10_Funet=res_10_Funet.F
F10_Abvt=res_10_Abvt.F
F10_Intellifiber=res_10_Intellifiber.F
F10_TataNld=res_10_TataNld.F
F10_Internode=res_10_Internode.F
F10_Missouri=res_10_Missouri.F
F10_Ion=res_10_Ion.F
F10_Palmetto=res_10_Palmetto.F

# # Nadir Point from 9 Algorithms: 
# ref_point = [1.81000000e+02 6.20000000e+01 3.92281875e+03 1.70537209e+00]
# # Nadir Point from 4 Algorithms: 
# ref_point = [160.  77.  inf  inf]
# # Nadir Point from all (13) Algorithms: 
# ref_point = [181.  77.  inf  inf]

ref_point = [181, 77, 3922.81875, 1.70537209]

hist_F10_Cogent = []
hist_F10_UsCarrier = []
hist_F10_HiberniaGlobal = []
hist_F10_Colt = []
hist_F10_Funet = []
hist_F10_Abvt = []
hist_F10_Intellifiber = []
hist_F10_TataNld = []
hist_F10_Internode = []
hist_F10_Missouri = []
hist_F10_Ion = []
hist_F10_Palmetto = []


for algo10_Cogent in res_10_Cogent.history:
    opt10_Cogent = algo10_Cogent.opt
    feas10_Cogent = np.where(opt10_Cogent.get("feasible"))[0]
    hist_F10_Cogent.append(opt10_Cogent.get("F")[feas10_Cogent])
for algo10_UsCarrier in res_10_UsCarrier.history:
    opt10_UsCarrier = algo10_UsCarrier.opt
    feas10_UsCarrier = np.where(opt10_UsCarrier.get("feasible"))[0]
    hist_F10_UsCarrier.append(opt10_UsCarrier.get("F")[feas10_UsCarrier])
for algo10_HiberniaGlobal in res_10_HiberniaGlobal.history:
    opt10_HiberniaGlobal = algo10_HiberniaGlobal.opt
    feas10_HiberniaGlobal = np.where(opt10_HiberniaGlobal.get("feasible"))[0]
    hist_F10_HiberniaGlobal.append(opt10_HiberniaGlobal.get("F")[feas10_HiberniaGlobal])
for algo10_Colt in res_10_Colt.history:
    opt10_Colt = algo10_Colt.opt
    feas10_Colt = np.where(opt10_Colt.get("feasible"))[0]
    hist_F10_Colt.append(opt10_Colt.get("F")[feas10_Colt])
for algo10_Funet in res_10_Funet.history:
    opt10_Funet = algo10_Funet.opt
    feas10_Funet = np.where(opt10_Funet.get("feasible"))[0]
    hist_F10_Funet.append(opt10_Funet.get("F")[feas10_Funet])
for algo10_Abvt in res_10_Abvt.history:
    opt10_Abvt = algo10_Abvt.opt
    feas10_Abvt = np.where(opt10_Abvt.get("feasible"))[0]
    hist_F10_Abvt.append(opt10_Abvt.get("F")[feas10_Abvt])
for algo10_Intellifiber in res_10_Intellifiber.history:
    opt10_Intellifiber = algo10_Intellifiber.opt
    feas10_Intellifiber = np.where(opt10_Intellifiber.get("feasible"))[0]
    hist_F10_Intellifiber.append(opt10_Intellifiber.get("F")[feas10_Intellifiber])
for algo10_TataNld in res_10_TataNld.history:
    opt10_TataNld = algo10_TataNld.opt
    feas10_TataNld = np.where(opt10_TataNld.get("feasible"))[0]
    hist_F10_TataNld.append(opt10_TataNld.get("F")[feas10_TataNld])
for algo10_Internode in res_10_Internode.history:
    opt10_Internode = algo10_Internode.opt
    feas10_Internode= np.where(opt10_Internode.get("feasible"))[0]
    hist_F10_Internode.append(opt10_Internode.get("F")[feas10_Internode])
for algo10_Missouri in res_10_Missouri.history:
    opt10_Missouri = algo10_Missouri.opt
    feas10_Missouri = np.where(opt10_Missouri.get("feasible"))[0]
    hist_F10_Missouri.append(opt10_Missouri.get("F")[feas10_Missouri])
for algo10_Ion in res_10_Ion.history:
    opt10_Ion = algo10_Ion.opt
    feas10_Ion = np.where(opt10_Ion.get("feasible"))[0]
    hist_F10_Ion.append(opt10_Ion.get("F")[feas10_Ion])
for algo10_Palmetto in res_10_Palmetto.history:
    opt10_Palmetto = algo10_Palmetto.opt
    feas10_Palmetto = np.where(opt10_Palmetto.get("feasible"))[0]
    hist_F10_Palmetto.append(opt10_Palmetto.get("F")[feas10_Palmetto])

2. Calculate hypervolume and merge them (Mean value) for ctaea algorithm

In [ ]:
from pymoo.indicators.hv import Hypervolume

metric = Hypervolume(ref_point= np.array(ref_point))

hv_10_Cogent = [metric.do(_F10_Cogent) for _F10_Cogent in hist_F10_Cogent]
hv_10_UsCarrier = [metric.do(_F10_UsCarrier) for _F10_UsCarrier in hist_F10_UsCarrier]
hv_10_HiberniaGlobal = [metric.do(_F10_HiberniaGlobal) for _F10_HiberniaGlobal in hist_F10_HiberniaGlobal]
hv_10_Colt = [metric.do(_F10_Colt) for _F10_Colt in hist_F10_Colt]
hv_10_Funet = [metric.do(_F10_Funet) for _F10_Funet in hist_F10_Funet]
hv_10_Abvt = [metric.do(_F10_Abvt) for _F10_Abvt in hist_F10_Abvt]
hv_10_Intellifiber = [metric.do(_F10_Intellifiber) for _F10_Intellifiber in hist_F10_Intellifiber]
hv_10_TataNld = [metric.do(_F10_TataNld) for _F10_TataNld in hist_F10_TataNld]
hv_10_Internode = [metric.do(_F10_Internode) for _F10_Internode in hist_F10_Internode]
hv_10_Missouri = [metric.do(_F10_Missouri) for _F10_Missouri in hist_F10_Missouri]
hv_10_Ion = [metric.do(_F10_Ion) for _F10_Ion in hist_F10_Ion]
hv_10_Palmetto = [metric.do(_F10_Palmetto) for _F10_Palmetto in hist_F10_Palmetto]

# ctaea
hv_10 = [(hv_Cogent + hv_UsCarrier + hv_HiberniaGlobal + hv_Colt + hv_Funet + hv_Abvt + hv_Intellifiber + hv_TataNld + hv_Internode + hv_Missouri + hv_Ion + hv_Palmetto) / 12 for hv_Cogent, hv_UsCarrier, hv_HiberniaGlobal, hv_Colt, hv_Funet, hv_Abvt, hv_Intellifiber, hv_TataNld, hv_Internode, hv_Missouri, hv_Ion, hv_Palmetto in zip(hv_10_Cogent, hv_10_UsCarrier, hv_10_HiberniaGlobal, hv_10_Colt, hv_10_Funet, hv_10_Abvt, hv_10_Intellifiber, hv_10_TataNld, hv_10_Internode, hv_10_Missouri, hv_10_Ion, hv_10_Palmetto)]

In [ ]:
print('The hypervolumn for Cogent:')
print(hv_10_Cogent)
print('The hypervolumn for UsCarrier:')
print(hv_10_UsCarrier)
print('The hypervolumn for HiberniaGlobal:')
print(hv_10_HiberniaGlobal)
print('The hypervolumn for Colt:')
print(hv_10_Colt)
print('The hypervolumn for Funet:')
print(hv_10_Funet)
print('The hypervolumn for Abvt:')
print(hv_10_Abvt)
print('The hypervolumn for Intellifiber:')
print(hv_10_Intellifiber)
print('The hypervolumn for TataNld:')
print(hv_10_TataNld)
print('The hypervolumn for Internode:')
print(hv_10_Internode)
print('The hypervolumn for Missouri:')
print(hv_10_Missouri)
print('The hypervolumn for Ion:')
print(hv_10_Ion)
print('The hypervolumn for Palmetto:')
print(hv_10_Palmetto)

# Show all hv for each topo
import matplotlib.pyplot as plt

plt.figure(figsize=(7, 5))
plt.plot(list(range(1, len(hv_10_Cogent)+1)), hv_10_Cogent,  color='black', label='Cogent')
plt.plot(list(range(1, len(hv_10_UsCarrier)+1)), hv_10_UsCarrier,  color='blue', label='UsCarrier')
plt.plot(list(range(1, len(hv_10_HiberniaGlobal)+1)), hv_10_HiberniaGlobal,  color='red', label='HiberniaGlobal')
plt.plot(list(range(1, len(hv_10_Colt)+1)), hv_10_Colt,  color='orange', label='Colt')
plt.plot(list(range(1, len(hv_10_Funet)+1)), hv_10_Funet,  color=(0, 0, 1, 0.5), label='Funet')
plt.plot(list(range(1, len(hv_10_Abvt)+1)), hv_10_Abvt,  color='gray', label='Abvt')
plt.plot(list(range(1, len(hv_10_Intellifiber)+1)), hv_10_Intellifiber,  color='purple', label='Intellifiber')
plt.plot(list(range(1, len(hv_10_TataNld)+1)), hv_10_TataNld,  color='green', label='TataNld')
plt.plot(list(range(1, len(hv_10_Internode)+1)), hv_10_Internode,  color='#33FF57', label='Internode')
plt.plot(list(range(1, len(hv_10_Missouri)+1)), hv_10_Missouri,  color='brown', label='Missouri')
plt.plot(list(range(1, len(hv_10_Ion)+1)), hv_10_Ion,  color='pink', label='Ion')
plt.plot(list(range(1, len(hv_10_Palmetto)+1)), hv_10_Palmetto,  color='olive', label='Palmetto')
plt.title("Convergence")
plt.xlabel("Topos")
plt.ylabel("Hypervolume")
plt.legend()
plt.show()

In [ ]:
print('The aveage of hypervolumn for ctaea:')
print(hv_10)

3. Draw mean hypervolume for ctaea

In [ ]:
import matplotlib.pyplot as plt

#  Show all alogrithms' hv into a single figure
plt.figure(figsize=(7, 5))
# plt.plot(list(range(1, len(hv_1)+1)), hv_1,  color='black', label='AGEMOEA')
# plt.plot(list(range(1, len(hv_2)+1)), hv_2,  color='blue', label='AGEMOEA2')
# plt.plot(list(range(1, len(hv_3)+1)), hv_3,  color='red', label='NSGA2')
# plt.plot(list(range(1, len(hv_4)+1)), hv_4,  color='orange', label='NSGA3')
# plt.plot(list(range(1, len(hv_5)+1)), hv_5,  color=(0, 0, 1, 0.5), label='RNSGA2')
# plt.plot(list(range(1, len(hv_6)+1)), hv_6,  color='gray', label='RNSGA3')
# plt.plot(list(range(1, len(hv_7)+1)), hv_7,  color='purple', label='RVEA')
# plt.plot(list(range(1, len(hv_8)+1)), hv_8,  color='green', label='SMSEMOA')
# plt.plot(list(range(1, len(hv_9)+1)), hv_9,  color='#33FF57', label='UNSGA3')
plt.plot(list(range(1, len(hv_10)+1)), hv_10,  color='brown', label='CTAEA')
# plt.plot(list(range(1, len(hv_11)+1)), hv_11,  color='pink', label='DNSGA2')
# plt.plot(list(range(1, len(hv_12)+1)), hv_12,  color='olive', label='KGBDMOEA')
# plt.plot(list(range(1, len(hv_13)+1)), hv_13,  color='cyan', label='MOEAD')
plt.title("Convergence")
plt.xlabel("Generations") # brown,pink,#5733FF,olive,cyan,#FF5733,#33FF57
plt.ylabel("Hypervolume")
plt.legend()
plt.show()